In [ ]:
import numpy as np
import matplotlib.pylab as plt
import glob
import time

## Load adversarial attacks and computer: Delta, Fourier Spectrum, L2 Distance and Maximum Amplitude of Fourier Spectrum

In [ ]:
def get_fourier(imag):
    image_sample = np.fft.fft2(imag)
    F_orig = np.fft.fftshift(image_sample)
    F_orig = np.log(np.abs(F_orig))
    return F_orig

In [ ]:
import scipy.spatial
image_total = []
delta_total = []
distance_total = []
fourier_total = []
maximum_total = []
original_total = []
adversarial_total = []
normal_total = []
m = time.time()
model = 'simplelinear'
for i in range(1, 938):
    name = './mnist_cnn/mnist_{}_10_{}_1_linf_adversarial.npy'.format(model, i)
    b = np.load(name, allow_pickle=True)
    b = b[0,:,:]
    normals = []
    images = []
    deltas = []
    distances = []
    fouriers = []
    maximums = []
    originals = []
    adversarials = []
    for j in range(b.shape[0]):
        if b[j,3] != None:
            normals.append(b[j,1])
            delta = b[j,0] - b[j,1]
            deltas.append(delta)
            fourier = get_fourier(delta)
            fouriers.append(fourier)
            maximums.append(fourier.max())
            original = b[j,1].flatten()
            adversarial = b[j,0].flatten()
            distance = scipy.spatial.distance.euclidean(original, adversarial)
            images.append(b[j,0])
            distances.append(distance)
            originals.append(b[j,2])
            adversarials.append(b[j,3])
        else:
            normals.append(np.zeros(b[j,0].shape))
            delta = np.zeros(b[j,0].shape)
            deltas.append(delta)
            images.append(delta)
            distances.append(None)
            fouriers.append(delta)
            maximums.append(None)
            originals.append(None)
            adversarials.append(None)
            
            
    images = np.array(images)
    deltas = np.array(deltas)
    distances = np.array(distances)
    fouriers = np.array(fouriers)
    maximums = np.array(maximums)
    originals = np.array(originals)
    adversarials = np.array(adversarials)
    image_total.append(images)
    delta_total.append(deltas)
    distance_total.append(distances)
    fourier_total.append(fouriers)
    maximum_total.append(maximums)
    original_total.append(originals)
    adversarial_total.append(adversarials)
    normal_total.append(normals)
    if i%25 ==0:
        print(i, time.time() - m)  
        m = time.time()

normal_total = np.array(normal_total)
image_total = np.array(image_total)
delta_total = np.array(delta_total)
distance_total = np.array(distance_total)
fourier_total = np.array(fourier_total)
maximum_total = np.array(maximum_total)
original_total = np.array(original_total)
adversarial_total = np.array(adversarial_total)



In [ ]:
np.save('./mnist_cnn/mnist_dynamics_adv_normal_total_{}.npy'.format(model), normal_total)
np.save('./mnist_cnn/mnist_dynamics_adv_image_total_{}.npy'.format(model), image_total)
np.save('./mnist_cnn/mnist_dynamics_adv_delta_total_{}.npy'.format(model), delta_total)
np.save('./mnist_cnn/mnist_dynamics_adv_distance_total_{}.npy'.format(model), distance_total)
np.save('./mnist_cnn/mnist_dynamics_adv_fourier_total_{}.npy'.format(model), fourier_total)
np.save('./mnist_cnn/mnist_dynamics_adv_maximum_total_{}.npy'.format(model), maximum_total)
np.save('./mnist_cnn/mnist_dynamics_adv_original_total_{}.npy'.format(model), original_total)
np.save('./mnist_cnn/mnist_dynamics_adv_adversarial_total_{}.npy'.format(model), adversarial_total)

## Main Visualization: 6 columns with 8 examples (one in each row). Visualizes [Original Image, Adversarial Image, Delta (normalize within a frame), Fourier Spectrum of the Delta. Maximum Amplitude through time, L2 Distance through Time]

In [ ]:
#%%capture
%matplotlib inline
import imageio
import io
import cv2
import random
#image_total, delta_total, distance_total, fourier_total, maximum_total, original_total, adversarial_total
import matplotlib.gridspec as gridspec
import time
def get_img_from_fig(fig, dpi=50):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img


def dyn_fourier(normal, image, delta, distance, fourier, maximum, original, adversarial, dist_max, max_max, idx, i):
    fig = plt.figure(figsize=(16, 16))
    #fig, axs = plt.subplots(8,5, figsize=(40, 20), facecolor='w', edgecolor='k')
    gs1 = gridspec.GridSpec(nrows=8, ncols=6, left=0.05, right=1.0,
    wspace=0.5, hspace=0.5)
    
    for j in range(8):

        axs = fig.add_subplot(gs1[j, 0])
        axs.imshow(normal[idx[j],0,:,:], vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))
        
        axs = fig.add_subplot(gs1[j, 1])
        axs.imshow(image[idx[j],0,:,:], vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 2])
        axs.imshow(delta[idx[j],0,:,:],vmin=0.0,vmax=1.0, interpolation='nearest')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 3])
        axs.imshow(fourier[idx[j],0,:,:],vmin=0.0, interpolation='nearest')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 4])
        axs.plot(maximum[:,idx[j]])
        axs.set_ylim(0,max_max)
        axs.set_ylabel('Maximum')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 5])
        axs.plot(distance[:,idx[j]])
        axs.set_ylim(0,dist_max)
        axs.set_ylabel('L2 Norm')
        axs.set_title('I: {}, C: {}. A: {}'.format(i,original[idx[j]], adversarial[idx[j]]))
    image = get_img_from_fig(fig)
    del fig
    return image

vmax = image_total.max()
kargs = { 'macro_block_size': None }
idx = [i for i in range(image_total[0].shape[0])]
random.shuffle(idx)
lss = []

for i in range(937):
    h = time.time()
    mm = dyn_fourier(normal_total[i,...], image_total[i,...], delta_total[i,...], 
            distance_total, fourier_total[i,...], 
            maximum_total, original_total[i,...], adversarial_total[i,...], distance_total[distance_total !=None].max(), maximum_total[maximum_total !=None].max(),
                     idx,i)
    lss.append(mm)
    print(i, time.time() - h)
    
#I save it because for some reason sometimes it runs out of memory at this point
np.save('./mnist_cnn/mnist_dynamics_{}.npy'.format(model), lss)


In [ ]:
imageio.mimsave('./mnist_cnn/image_total_dynamics_mnist_{}.mp4'.format(model), lss, fps=60, **kargs)